# 分词部分

In [1]:
import os 
import re
import jieba
import json
import pandas as pd

In [2]:
### ONLY FOR TEST
def test_segmentation(poem, user_dict=None):   
    for s in poem:
        seg_list = jieba.cut(s, cut_all=False)  
        print('/'.join(list(seg_list)))
        
def run_test_1(user_dict=None):
    poem = ['秦川雄帝宅，函谷壯皇居。', '綺殿千尋起，離宮百雉餘。', '連甍遙接漢，飛觀迥凌虛。', '雲日隱層闕，風煙出綺疎。']
    print('Poem #1')
    test_segmentation(poem, user_dict)
    
    poem = ['遠山媚平楚，宿雨漲清溪。', '沿泝任舟檝，歡言無町畦。', '夜深忽梦少年事，梦啼妆泪红阑干']
    print('Poem #2')
    test_segmentation(poem, user_dict)
    
    poem = ['剑外忽传收蓟北，初闻涕泪满衣裳。',
            '却看妻子愁何在，漫卷诗书喜欲狂。',
            '白日放歌须纵酒，青春作伴好还乡。',
            '即从巴峡穿巫峡，便下襄阳向洛阳。']
    print('Poem #3')
    test_segmentation(poem, user_dict)
    
    poem = ['明月几时有，把酒问青天。', '不知天上宫阙，今夕是何年。', '我欲乘风归去，又恐琼楼玉宇，高处不胜寒。', '起舞弄清影，何似在人间。',
              '转朱阁，低绮户，照无眠。', '不应有恨，何事长向别时圆。', '人有悲欢离合，月有阴晴圆缺，此事古难全。', '但愿人长久，千里共婵娟。']
    print('Poem #4')
    test_segmentation(poem, user_dict)

In [3]:
LYRIC_DIR = './ci/'
POEM_DIR = './json/'

## 唐诗

先测试一下jieba分词的原始分词功能，以下诗句明显分错：
+ 秦川/雄帝/宅/
+ 沿/泝/任舟/檝/，/歡言/無/町/畦/
+ /月/有/阴晴圆/缺/

In [4]:
run_test_1()

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/p6/kb71f5k967ldvqs9b6zb82qw0000gn/T/jieba.cache


Poem #1


Loading model cost 0.706 seconds.
Prefix dict has been built successfully.


秦川/雄帝/宅/，/函谷壯/皇居/。
綺/殿/千尋起/，/離宮/百雉/餘/。
連/甍/遙/接漢/，/飛觀/迥凌虛/。
雲日/隱層/闕/，/風煙出/綺/疎/。
Poem #2
遠山媚/平楚/，/宿雨漲/清溪/。
沿/泝/任舟/檝/，/歡言/無/町/畦/。
夜深/忽梦/少年/事/，/梦啼妆泪/红/阑干
Poem #3
剑外/忽/传收/蓟北/，/初闻/涕泪/满/衣裳/。
却/看/妻子/愁/何在/，/漫卷/诗书/喜欲狂/。
白日/放歌/须/纵酒/，/青春/作伴/好/还乡/。
即/从/巴峡/穿/巫峡/，/便/下/襄阳/向/洛阳/。
Poem #4
明月/几时/有/，/把酒/问青天/。
不知/天上/宫阙/，/今夕/是/何年/。
我/欲/乘风/归去/，/又/恐/琼楼玉宇/，/高处不胜/寒/。
起舞/弄清/影/，/何似/在/人间/。
转朱阁/，/低绮户/，/照无眠/。
不/应有/恨/，/何事/长向/别时圆/。
人有/悲欢离合/，/月/有/阴晴圆/缺/，/此事/古难全/。
但愿人长久/，/千里/共/婵娟/。


接下来我的想法是，先根据五字句的前两个字和七字句的前四个字构建词典，然后以此作为分词词典对整句进行分词。

我用了全唐诗和全宋诗作为数据集来构建词典。经测试，比只使用全唐诗更加精确。

In [5]:
poems = []
file_list = os.listdir(POEM_DIR)
for file in file_list:
    if file.startswith('poet.'):
        poems.append(pd.read_json(f'{POEM_DIR}{file}'))
poem_df = pd.concat(poems, ignore_index=True)

In [6]:
def cut_lines(poem):
    lines = []
    pattern = r',|\.|/|;|\'|`|\[|\]|<|>|\?|:|"|\{|\}|\~|!|@|#|\$|%|\^|&|\(|\)|-|=|\_|\+|，|。|、|；|‘|’|【|】|·|！| |…|（|）'
    for pairs in poem:
        lines.extend(list(filter(lambda x: len(x) > 0, re.split(pattern, pairs))))
    return lines

In [7]:
poem_df['lines'] = poem_df['paragraphs'].apply(cut_lines)

In [8]:
def first_segment(lines):
    words = []
    for line in lines:
        if len(line) == 5: 
            words.append(line[0:2])
        if len(line) == 7: 
            words.append(line[0:2])
            words.append(line[2:4])
    return words

In [9]:
poem_df['dic'] = poem_df['lines'].apply(first_segment)

In [10]:
word_freq = dict()

dictionary = [item for sublist in poem_df['dic'] for item in sublist]
for i in dictionary:
    word_freq[i] = word_freq.get(i, 0) + 1

word_freq = {key:val for key, val in word_freq.items() if val >= 3}

with open('./poem_dict.txt', 'w', encoding='utf-8') as f:
    for k, v in word_freq.items():
        f.write(f'{k} {v}\n')

jieba.set_dictionary('./poem_dict.txt')

测试目前词典的分词效果。可见之前使用原始词典分错的地方都能分对，但是也添加了几处错误：
+ /梦啼妆泪/红/阑/干
+ /初闻涕/泪满/衣裳/
+ 我欲/乘风/归/去/

但是目前我们只关心唐诗的五字句和七字句的后三个字的分词，所以可以认为效果明显变好，毕竟“雄/帝宅”，“任/舟檝”，“無/町畦”等都能分对。

In [11]:
run_test_1()

Building prefix dict from /Users/lihangqian/Documents/08洛桑联邦理工/lectures/History and the digital/history-digital-main/poem_dict.txt ...


Poem #1


Dumping model to file cache /var/folders/p6/kb71f5k967ldvqs9b6zb82qw0000gn/T/jieba.u04e80a0a4359adea0c01661aca1641c1.cache
Loading model cost 0.322 seconds.
Prefix dict has been built successfully.


秦川/雄/帝宅/，/函谷/壯/皇居/。
綺/殿/千尋/起/，/離宮/百雉/餘/。
連甍/遙接漢/，/飛觀/迥/凌虛/。
雲日/隱層/闕/，/風煙/出/綺疎/。
Poem #2
遠山/媚/平楚/，/宿雨/漲/清溪/。
沿泝/任/舟檝/，/歡言/無/町畦/。
夜深/忽梦/少年/事/，/梦啼妆泪/红/阑/干
Poem #3
剑外/忽/传收/蓟北/，/初闻涕/泪满/衣裳/。
却看/妻子/愁/何在/，/漫卷/诗书/喜欲狂/。
白日/放歌/须/纵酒/，/青春/作伴/好/还/乡/。
即/从/巴峡/穿/巫峡/，/便下/襄阳/向/洛阳/。
Poem #4
明月/几时/有/，/把酒/问/青天/。
不知/天上/宫阙/，/今夕/是何/年/。
我欲/乘风/归/去/，/又恐/琼楼/玉宇/，/高处/不胜寒/。
起舞/弄/清影/，/何似/在人/间/。
转朱阁/，/低绮户/，/照无眠/。
不应/有恨/，/何事/长向别/时/圆/。
人有/悲欢/离合/，/月有/阴晴/圆缺/，/此事/古难全/。
但愿/人长/久/，/千里/共婵娟/。


之后根据词典对后三个字再次进行分词，并据此扩充词典：

In [12]:
def addition_segment(phrase):
    result = jieba.lcut(phrase, cut_all=False)     
    if len(result) != 2:
        return phrase
    if len(result[0]) == 2:
        return result[0]
    return result[1]

def second_segment(lines):
    words = []
    for line in lines:
        if len(line) == 5:
            words.append(line[0:2])
            words.append(addition_segment(line[2:]))
        if len(line) == 7:
            words.append(line[0:2])
            words.append(line[2:4])
            words.append(addition_segment(line[4:]))
    return words

In [13]:
poem_df['dic_total'] = poem_df['lines'].apply(second_segment)

In [14]:
word_freq = dict()
dictionary = [item for sublist in poem_df['dic_total'] for item in sublist]
for i in dictionary:
    word_freq[i] = word_freq.get(i, 0) + 1

word_freq = {key:val for key, val in word_freq.items() if val >= 3}

with open('./whole_poem_dict.txt', 'w', encoding='utf-8') as f:
    for k, v in word_freq.items():
        f.write(f'{k} {v}\n')

jieba.set_dictionary('./whole_poem_dict.txt')

第二次的分词效果如下，可见效果再次变好，我觉得和上次的分词效果相当，虽然修正了“初闻/涕泪/满/衣裳”，但是“却看/妻子/愁何/在”又分错了。

In [15]:
run_test_1()

Building prefix dict from /Users/lihangqian/Documents/08洛桑联邦理工/lectures/History and the digital/history-digital-main/whole_poem_dict.txt ...


Poem #1


Dumping model to file cache /var/folders/p6/kb71f5k967ldvqs9b6zb82qw0000gn/T/jieba.u451ede831b0b3f802f1e06d231cf5503.cache
Loading model cost 0.371 seconds.
Prefix dict has been built successfully.


秦川/雄/帝宅/，/函谷/壯/皇居/。
綺/殿/千尋/起/，/離宮/百雉/餘/。
連甍/遙接漢/，/飛觀/迥/凌虛/。
雲日/隱層/闕/，/風煙/出/綺疎/。
Poem #2
遠山/媚/平楚/，/宿雨/漲/清溪/。
沿泝/任/舟檝/，/歡言/無/町畦/。
夜深/忽梦/少年/事/，/梦啼妆泪/红/阑/干
Poem #3
剑外/忽/传收/蓟北/，/初闻/涕泪/满/衣裳/。
却看/妻子/愁何/在/，/漫卷/诗书/喜欲狂/。
白日/放歌/须/纵酒/，/青春/作伴/好/还/乡/。
即/从/巴峡/穿/巫峡/，/便下/襄阳/向/洛阳/。
Poem #4
明月/几时/有/，/把酒/问/青天/。
不知/天上/宫阙/，/今夕/是何/年/。
我欲/乘风/归/去/，/又恐/琼楼/玉宇/，/高处/不胜寒/。
起舞/弄/清影/，/何似/在人/间/。
转朱阁/，/低绮户/，/照无眠/。
不应/有恨/，/何事/长向别/时/圆/。
人有/悲欢/离合/，/月有/阴晴/圆缺/，/此事/古难全/。
但愿/人长/久/，/千里/共婵娟/。


总结，jieba分词很有问题，词典中单词的频率并不能有效改善分词结果。比如在自定义的词典中，明明没有“愁何”，而且“何在”出现了很多次，但jieba仍然分成了“愁何/在”。

jieba的这个问题已经讨论了好几年了：
https://github.com/fxsjy/jieba/issues/14

## 宋词

我本想人工划分意向，但有些名句出现得太例外：
+ 六字句通常划分方式各异，最常见的为222式，但有的名句与众不同：七八个星天外，两三点雨山前。
+ “满江红”此句与别句的划分方式都不同：“驾长车，踏破贺兰山缺”，加之“贺兰山”是地名，故最好不要人工划分。

所以只能重回之前用的套路，先提取五字句中的前两个字和七字句中的前四字扩充词典，再根据扩充后的词典分词。

In [16]:
lyrics = []
file_list = os.listdir(LYRIC_DIR)
for file in file_list:
    if file.startswith('ci.song'):
        lyrics.append(pd.read_json(f'{LYRIC_DIR}{file}'))
lyric_df = pd.concat(lyrics, ignore_index=True)
lyric_df

,author,paragraphs,rhythmic,tags,prologue
0,张榘,"[碧油推上客。, 有神机沈密，参运帷幄。, 威声际沙漠。, 庆云飞川泳，和熏三白。, 霄渊鬲...",瑞鹤仙,NaN,NaN
1,张榘,"[绿野归来，筇杖角巾，岂不快哉。, 有清泉白石，东西岩岫，翠阴红影，高下楼台。, 况是蕤宾，...",沁园春,NaN,NaN
2,张榘,"[豆花轻雨霁，更七日、是中秋。, 记分野三台，家山双阙，孕秀名流。, 平生佐时大略，有忠勤、...",木兰花慢,NaN,NaN
3,张榘,"[素壁走龙蛇，难觅醉翁真迹。, 惟有断岗衰草，是几番经历。, 紫萸黄菊又西风，同作携壶客。,...",好事近,NaN,NaN
4,张榘,"[望神京、目断烟草，青天长剑频倚。, 香街十里朱帘月，空想当年华丽。, 堪叹处。, 渺沙霭蒹...",摸鱼儿,NaN,NaN
...,...,...,...,...,...
21048,楼钥,"[冷然。, 清圆。, 谁弹。, 向屋山。, 何言。, 清风至阴德之天。, 悠馀响婵娟。, 方...",醉翁操,NaN,NaN
21049,楼钥,"[孝宗纯孝，前圣更何加。, 高蹈处重华。, 丹成仙去龙輴远，越岸暮山遐。, 波臣先为卷寒沙。...",导引,NaN,NaN
21050,楼钥,"[吾皇尽孝，宗庙务崇尊。, 巨典备弥文。, 巍巍东向开基主，七世神孙。, 追思九闰整乾坤。,...",导引,NaN,NaN
21051,张良臣,"[昨日豆花篱下过，忽然迎面好风吹。, 独自立多时。]",失调名,NaN,NaN


词牌名概览。出现次数最多的15个词牌名占了宋词的三分之一：

In [17]:
rhythm_df = lyric_df.groupby(['rhythmic']).count()
rhythm_df.sort_values(by='paragraphs', ascending=False).head(15)

,author,paragraphs,tags,prologue
rhythmic,,,,
浣溪沙,776,776,16,1
水调歌头,743,743,45,0
鹧鸪天,657,657,19,0
菩萨蛮,598,598,9,0
满江红,549,549,30,0
西江月,490,490,7,2
临江仙,482,482,13,0
念奴娇,432,432,28,0
减字木兰花,425,425,4,0


In [18]:
lyric_df['lines'] = lyric_df['paragraphs'].apply(cut_lines)

In [19]:
lyric_df['dic'] = lyric_df['lines'].apply(first_segment)
lyric_df['dic']

0                            [碧油, 有神, 威声, 庆云, 更谁, 看鹅]
1                [有清, 照眼, 葵榴, 翦香, 看看, 算支, 应有, 天边, 为苍]
2                        [豆花, 记分, 一念, 须知, 万灶, 智勇, 喜色]
3                            [素壁, 是几, 紫萸, 黄菊, 同作, 负晴]
4                [香街, 十里, 渺沙, 咿ン, 怅杨, 陈迹, 看玉, 笑骋, 麟阁]
                             ...                     
21048    [清风, 至阴, 悠馀, 迥立, 八音, 相宣, 聊将, 听之, 弦索, 未有, 逸韵]
21049    [前圣, 高蹈, 丹成, 仙去, 越岸, 波臣, 先为, 来往, 九虞, 礼举, 万世]
21050        [宗庙, 巨典, 巍巍, 东向, 追思, 九闰, 环宇, 从今, 密迩, 千载]
21051                            [昨日, 豆花, 忽然, 迎面, 独自]
21052                                [殷云, 度雨, 蛮江, 豆蔻]
Name: dic, Length: 21053, dtype: object

In [20]:
word_freq = dict()
dictionary = [item for sublist in poem_df['dic'] for item in sublist]
lyric_dictionary = [item for sublist in lyric_df['dic'] for item in sublist]
dictionary.extend(lyric_dictionary)
for i in dictionary:
    word_freq[i] = word_freq.get(i, 0) + 1

word_freq = {key:val for key, val in word_freq.items() if val >= 3}
    
with open('./lyric_dict.txt', 'w', encoding='utf-8') as f:
    for k, v in word_freq.items():
        f.write(f'{k} {v * 100}\n')

jieba.set_dictionary('./lyric_dict.txt')

In [21]:
def lyric_second_segment(lines):
    words = []
    for line in lines:
        if len(line) == 5:
            words.append(line[0:2])
            words.append(addition_segment(line[2:]))
        if len(line) == 7:
            words.append(line[0:2])
            words.append(line[2:4])
            words.append(addition_segment(line[4:]))
        else:
            tmp = jieba.lcut(line, cut_all=False)
            words.extend(list(filter(lambda x: len(x) >= 2, tmp)))
    return words

In [22]:
lyric_df['dic_total'] = lyric_df['lines'].apply(lyric_second_segment)

Building prefix dict from /Users/lihangqian/Documents/08洛桑联邦理工/lectures/History and the digital/history-digital-main/lyric_dict.txt ...
Dumping model to file cache /var/folders/p6/kb71f5k967ldvqs9b6zb82qw0000gn/T/jieba.u1ac65e3ace2d1afb4d2a899ff1209d9e.cache
Loading model cost 0.332 seconds.
Prefix dict has been built successfully.


In [23]:
run_test_1()

Poem #1
秦川/雄/帝宅/，/函谷/壯/皇居/。
綺/殿/千尋/起/，/離宮/百雉/餘/。
連甍/遙接漢/，/飛觀/迥/凌虛/。
雲日/隱層/闕/，/風煙/出/綺疎/。
Poem #2
遠山/媚/平楚/，/宿雨/漲/清溪/。
沿泝/任/舟檝/，/歡言/無/町畦/。
夜深/忽梦/少年/事/，/梦/啼妆/泪红/阑干
Poem #3
剑外/忽/传收/蓟北/，/初闻/涕泪/满/衣裳/。
却看/妻子/愁/何在/，/漫卷/诗书/喜欲狂/。
白日/放歌/须/纵酒/，/青春/作伴/好/还/乡/。
即/从/巴峡/穿/巫峡/，/便下/襄阳/向/洛阳/。
Poem #4
明月/几时/有/，/把酒/问/青天/。
不知/天上/宫阙/，/今夕/是何/年/。
我欲/乘风/归去/，/又恐/琼楼/玉宇/，/高处/不胜/寒/。
起舞/弄/清影/，/何似/在人/间/。
转/朱阁/，/低绮户/，/照/无眠/。
不应/有恨/，/何事/长向/别时/圆/。
人有/悲欢/离合/，/月有/阴晴/圆缺/，/此事/古难全/。
但愿/人长/久/，/千里/共/婵娟/。


最后一次分词的结果。“却看/妻子/愁/何在”终于分对了，虽然“梦/啼妆/泪红/阑干”分错，但是我们只关心对后三个字的分词，所以无伤大雅。最关键的是，“转/朱阁”终于分出来了，“千里/共/婵娟”的“婵娟”也分出来了。但是上述几个分词词典都没正确地分出“但愿/人长/久”，我看字典里没有“长久”和“人长”，所以我认为是它随便分了。

总结一下我的想法：

假设：五字句AABBB的前两个字AA，七字句AABBCCC的前四个字AA和BB，统一视为可以直接提取出来的“好意象”。当然这个规律也有例外，但是我只找到白居易的“家在虾蟆陵下住”这一个反例，我觉得只是因为白居易本来就专门写通俗化口语化的诗。后三个字的分法大相径庭，所以需要借助分词工具。

做法：
1. 先用唐诗和宋诗的“好意象”构建词典，然后用得到的词典对五字句AABBB的后三个字BBB，七字句AABBCCC的后三个字CCC分词。用对BBB和CCC的分词结果扩充词典。
2. 唐诗的分词完成。分词结果就是好意象+后三个字的分词。
3. 用宋词里的五字句的和七字句的“好意象”扩充词典，对宋词其他句式进行分词，
4. 宋词的分词完成。分词结果就是：
        对于五字句和七字句：好意象+后三个字的分词；
        对于其他句式：整句的分词结果

In [24]:
song_df = lyric_df

poems = []
file_list = os.listdir(POEM_DIR)
for file in file_list:
    if file.startswith('poet.tang'):
        poems.append(pd.read_json(f'{POEM_DIR}{file}'))
tang_df = pd.concat(poems, ignore_index=True)
tang_df['lines'] = tang_df['paragraphs'].apply(cut_lines)
jieba.set_dictionary('./poem_dict.txt')
tang_df['dic_total'] = tang_df['lines'].apply(second_segment)
jieba.set_dictionary('./lyric_dict.txt')

Building prefix dict from /Users/lihangqian/Documents/08洛桑联邦理工/lectures/History and the digital/history-digital-main/poem_dict.txt ...
Loading model from cache /var/folders/p6/kb71f5k967ldvqs9b6zb82qw0000gn/T/jieba.u04e80a0a4359adea0c01661aca1641c1.cache
Loading model cost 0.220 seconds.
Prefix dict has been built successfully.


总之，目前可以使用tang_df['dic_total']和song_df['dic_total']来看唐诗和宋词的分词结果：

In [25]:
tang_df

,author,paragraphs,title,id,tags,lines,dic_total
0,陳嘉言,"[公子申敬愛，攜朋翫物華。, 人是平陽客，地即石崇家。, 水文生舊浦，風色滿新花。, 日暮連...",晦日宴高氏林亭,30a199ac-e508-46c2-b3ed-ecbcda4e4d81,NaN,"[公子申敬愛, 攜朋翫物華, 人是平陽客, 地即石崇家, 水文生舊浦, 風色滿新花, 日暮連...","[公子, 敬愛, 攜朋, 翫物, 人是, 平陽, 地即, 石崇, 水文, 舊浦, 風色, 新..."
1,陳嘉言,"[高門引冠蓋，下客抱支離。, 綺席珍羞滿，文場翰藻摛。, 蓂華彫上月，柳色藹春池。, 日斜歸...",晦日重宴,d3098b52-2a7b-4e81-9afc-af4f6ed53817,NaN,"[高門引冠蓋, 下客抱支離, 綺席珍羞滿, 文場翰藻摛, 蓂華彫上月, 柳色藹春池, 日斜歸...","[高門, 冠蓋, 下客, 支離, 綺席, 珍羞, 文場, 翰藻, 蓂華, 上月, 柳色, 春..."
2,陳嘉言,"[今夜可憐春，河橋多麗人。, 寶馬金爲絡，香車玉作輪。, 連手窺潘掾，分頭看洛神。, 重城自...",上元夜效小庾體,fb0abe69-b0e5-40d9-a852-5bddc6c38abf,NaN,"[今夜可憐春, 河橋多麗人, 寶馬金爲絡, 香車玉作輪, 連手窺潘掾, 分頭看洛神, 重城自...","[今夜, 可憐, 河橋, 麗人, 寶馬, 金爲, 香車, 玉作, 連手, 窺潘掾, 分頭, ..."
3,周彥暉,"[砌蓂收晦魄，津柳競年華。, 既狎忘筌友，方淹投轄車。, 綺筵迴舞雪，瓊醑泛流霞。, 雲低上...",晦日宴高氏林亭,d756f172-bcc6-4835-9efa-e3adc7c7d939,NaN,"[砌蓂收晦魄, 津柳競年華, 既狎忘筌友, 方淹投轄車, 綺筵迴舞雪, 瓊醑泛流霞, 雲低上...","[砌蓂, 收晦, 津柳, 年華, 既狎, 忘筌, 方淹, 投轄, 綺筵, 舞雪, 瓊醑, 流..."
4,周彥暉,"[春華歸柳樹，俯景落蓂枝。, 置驛銅街右，開筵玉浦陲。, 林煙含障密，竹雨帶珠危。, 興闌巾...",晦日重宴,bcb57f9c-5b88-42f7-8316-af64a2c96c8f,NaN,"[春華歸柳樹, 俯景落蓂枝, 置驛銅街右, 開筵玉浦陲, 林煙含障密, 竹雨帶珠危, 興闌巾...","[春華, 柳樹, 俯景, 落蓂枝, 置驛, 銅街, 開筵, 玉浦陲, 林煙, 含障, 竹雨,..."
...,...,...,...,...,...,...,...
57607,耿湋,"[寥寂荒壘下，客舍雨微微。, 門見苔生滿，心慙吏到稀。, 籬花看未發，海燕欲先歸。, 無限堪...",盩厔客舍,96d8b98d-6b4a-4eb0-b915-988296f67cd3,NaN,"[寥寂荒壘下, 客舍雨微微, 門見苔生滿, 心慙吏到稀, 籬花看未發, 海燕欲先歸, 無限堪...","[寥寂, 荒壘, 客舍, 雨微, 門見, 苔生, 心慙, 吏到, 籬花, 未發, 海燕, 欲..."
57608,耿湋,"[傳經韋相後，賜筆漢家郎。, 幽閣諸生會，寒宵幾刻長。, 座中燈泛酒，簷外月如霜。, 人事多...",宿韋員外宅,94209809-403f-4bb0-9fef-04b603bed685,NaN,"[傳經韋相後, 賜筆漢家郎, 幽閣諸生會, 寒宵幾刻長, 座中燈泛酒, 簷外月如霜, 人事多...","[傳經, 韋相, 賜筆, 漢家, 幽閣, 諸生, 寒宵, 幾刻, 座中, 泛酒, 簷外, 月..."
57609,耿湋,"[沃州初望海，攜手盡時髦。, 小暑開鵬翼，新蓂長鷺濤。, 月如芳草遠，身比夕陽高。, 羊祜傷...",登沃州山,e9b8408c-490a-4ea1-a458-f3249bb7ddaf,NaN,"[沃州初望海, 攜手盡時髦, 小暑開鵬翼, 新蓂長鷺濤, 月如芳草遠, 身比夕陽高, 羊祜傷...","[沃州, 望海, 攜手, 盡時, 小暑, 鵬翼, 新蓂, 鷺濤, 月如, 芳草, 身比, 夕..."
57610,耿湋,"[暝色休羣動，秋齋遠客情。, 細風和雨氣，寒竹度簾聲。, 日覺蹉跎近，天敎嬾慢成。, 誰能謁...",宿岐山姜明府廳,79ebb031-490c-44b2-b892-b779ace88f3a,NaN,"[暝色休羣動, 秋齋遠客情, 細風和雨氣, 寒竹度簾聲, 日覺蹉跎近, 天敎嬾慢成, 誰能謁...","[暝色, 羣動, 秋齋, 遠客, 細風, 和雨, 寒竹, 度簾聲, 日覺, 蹉跎, 天敎, ..."


In [26]:
song_df

,author,paragraphs,rhythmic,tags,prologue,lines,dic,dic_total
0,张榘,"[碧油推上客。, 有神机沈密，参运帷幄。, 威声际沙漠。, 庆云飞川泳，和熏三白。, 霄渊鬲...",瑞鹤仙,NaN,NaN,"[碧油推上客, 有神机沈密, 参运帷幄, 威声际沙漠, 庆云飞川泳, 和熏三白, 霄渊鬲, ...","[碧油, 有神, 威声, 庆云, 更谁, 看鹅]","[碧油, 推上, 碧油, 推上, 有神, 机沈密, 有神, 机沈密, 参运, 帷幄, 威声,..."
1,张榘,"[绿野归来，筇杖角巾，岂不快哉。, 有清泉白石，东西岩岫，翠阴红影，高下楼台。, 况是蕤宾，...",沁园春,NaN,NaN,"[绿野归来, 筇杖角巾, 岂不快哉, 有清泉白石, 东西岩岫, 翠阴红影, 高下楼台, 况是...","[有清, 照眼, 葵榴, 翦香, 看看, 算支, 应有, 天边, 为苍]","[绿野, 归来, 筇杖, 角巾, 不快, 有清, 白石, 有清, 白石, 东西, 翠阴, 红..."
2,张榘,"[豆花轻雨霁，更七日、是中秋。, 记分野三台，家山双阙，孕秀名流。, 平生佐时大略，有忠勤、...",木兰花慢,NaN,NaN,"[豆花轻雨霁, 更七日, 是中秋, 记分野三台, 家山双阙, 孕秀名流, 平生佐时大略, 有...","[豆花, 记分, 一念, 须知, 万灶, 智勇, 喜色]","[豆花, 雨霁, 豆花, 雨霁, 七日, 是中, 记分, 三台, 分野, 三台, 家山, 双..."
3,张榘,"[素壁走龙蛇，难觅醉翁真迹。, 惟有断岗衰草，是几番经历。, 紫萸黄菊又西风，同作携壶客。,...",好事近,NaN,NaN,"[素壁走龙蛇, 难觅醉翁真迹, 惟有断岗衰草, 是几番经历, 紫萸黄菊又西风, 同作携壶客,...","[素壁, 是几, 紫萸, 黄菊, 同作, 负晴]","[素壁, 走龙蛇, 素壁, 走龙蛇, 难觅, 醉翁, 真迹, 惟有, 断岗, 衰草, 是几,..."
4,张榘,"[望神京、目断烟草，青天长剑频倚。, 香街十里朱帘月，空想当年华丽。, 堪叹处。, 渺沙霭蒹...",摸鱼儿,NaN,NaN,"[望神京, 目断烟草, 青天长剑频倚, 香街十里朱帘月, 空想当年华丽, 堪叹处, 渺沙霭蒹...","[香街, 十里, 渺沙, 咿ン, 怅杨, 陈迹, 看玉, 笑骋, 麟阁]","[神京, 目断, 烟草, 青天, 长剑, 频倚, 香街, 十里, 朱帘, 空想, 当年, 华..."
...,...,...,...,...,...,...,...,...
21048,楼钥,"[冷然。, 清圆。, 谁弹。, 向屋山。, 何言。, 清风至阴德之天。, 悠馀响婵娟。, 方...",醉翁操,NaN,NaN,"[冷然, 清圆, 谁弹, 向屋山, 何言, 清风至阴德之天, 悠馀响婵娟, 方昼眠, 迥立八...","[清风, 至阴, 悠馀, 迥立, 八音, 相宣, 聊将, 听之, 弦索, 未有, 逸韵]","[冷然, 清圆, 屋山, 何言, 清风, 至阴, 德之, 悠馀, 婵娟, 悠馀响, 婵娟, ..."
21049,楼钥,"[孝宗纯孝，前圣更何加。, 高蹈处重华。, 丹成仙去龙輴远，越岸暮山遐。, 波臣先为卷寒沙。...",导引,NaN,NaN,"[孝宗纯孝, 前圣更何加, 高蹈处重华, 丹成仙去龙輴远, 越岸暮山遐, 波臣先为卷寒沙, ...","[前圣, 高蹈, 丹成, 仙去, 越岸, 波臣, 先为, 来往, 九虞, 礼举, 万世]","[孝宗纯, 前圣, 更何, 前圣, 更何, 高蹈, 重华, 高蹈, 重华, 丹成, 仙去, ..."
21050,楼钥,"[吾皇尽孝，宗庙务崇尊。, 巨典备弥文。, 巍巍东向开基主，七世神孙。, 追思九闰整乾坤。,...",导引,NaN,NaN,"[吾皇尽孝, 宗庙务崇尊, 巨典备弥文, 巍巍东向开基主, 七世神孙, 追思九闰整乾坤, 环...","[宗庙, 巨典, 巍巍, 东向, 追思, 九闰, 环宇, 从今, 密迩, 千载]","[吾皇, 尽孝, 宗庙, 务崇尊, 宗庙务崇尊, 巨典, 备弥文, 巨典, 备弥文, 巍巍,..."
21051,张良臣,"[昨日豆花篱下过，忽然迎面好风吹。, 独自立多时。]",失调名,NaN,NaN,"[昨日豆花篱下过, 忽然迎面好风吹, 独自立多时]","[昨日, 豆花, 忽然, 迎面, 独自]","[昨日, 豆花, 篱下, 忽然, 迎面, 好风, 独自, 多时, 独自, 多时]"


In [27]:
tang_df.to_csv("tang_seg.csv")

In [28]:
song_df.to_csv("song_seg.csv")

In [29]:
libai = tang_df[tang_df['author']== '李白']
li_seg = libai['dic_total'].tolist()

In [30]:
li_seg[0]

['三十',
 '離宮',
 '樓臺',
 '與天',
 '閣道',
 '步行',
 '美人',
 '愁煙',
 '恩疎',
 '不及',
 '桃李',
 '傷春',
 '淫樂',
 '意何極',
 '金輿',
 '回中',
 '萬乘',
 '黃道',
 '千旗',
 '彩虹',
 '前軍',
 '細柳',
 '後騎',
 '甘泉',
 '豈問',
 '渭川',
 '寧邀',
 '野童',
 '秋暮',
 '瑤池',
 '歸來',
 '未窮']

In [31]:
import operator
from functools import reduce

li_list = reduce(operator.add, li_seg)
li_list

['三十',
 '離宮',
 '樓臺',
 '與天',
 '閣道',
 '步行',
 '美人',
 '愁煙',
 '恩疎',
 '不及',
 '桃李',
 '傷春',
 '淫樂',
 '意何極',
 '金輿',
 '回中',
 '萬乘',
 '黃道',
 '千旗',
 '彩虹',
 '前軍',
 '細柳',
 '後騎',
 '甘泉',
 '豈問',
 '渭川',
 '寧邀',
 '野童',
 '秋暮',
 '瑤池',
 '歸來',
 '未窮',
 '洗兵',
 '條支',
 '海上',
 '放馬',
 '天山',
 '雪中',
 '萬里',
 '長征',
 '三軍',
 '衰老',
 '秦家',
 '築城',
 '備胡處',
 '漢家',
 '還有',
 '烽火',
 '烽火',
 '不息',
 '征戰',
 '無已',
 '野戰',
 '格鬬',
 '敗馬',
 '號鳴',
 '向天',
 '烏鳶',
 '人腸',
 '銜飛',
 '上挂',
 '枯樹',
 '士卒',
 '草莽',
 '將軍',
 '空爾',
 '乃知',
 '兵者',
 '凶器',
 '奔流',
 '到海',
 '不復',
 '朝如',
 '青絲',
 '成雪',
 '人生',
 '得意',
 '須盡',
 '莫使',
 '金尊',
 '空對',
 '天生',
 '我材',
 '必有',
 '千金',
 '散盡',
 '還復',
 '烹羊',
 '宰牛',
 '且爲',
 '會須',
 '一飲',
 '三百',
 '與君',
 '一曲',
 '請君',
 '爲我',
 '側耳',
 '鐘鼓',
 '饌玉',
 '不足',
 '但願',
 '長醉',
 '不復',
 '古來',
 '聖賢',
 '寂寞',
 '惟有',
 '飲者',
 '其名',
 '陳王',
 '昔時',
 '平樂',
 '斗酒',
 '十千',
 '恣歡',
 '主人',
 '何爲',
 '言少',
 '徑須',
 '酤取',
 '對君',
 '呼兒',
 '將出',
 '美酒',
 '與爾',
 '同銷',
 '萬古',
 '馬色',
 '雖不',
 '人心',
 '本無',
 '共作',
 '遊冶',
 '雙行',
 '洛陽',
 '長劒',
 '照曜',
 '高冠',
 '何赩赫',
 '各有',
 '千金',
 '俱

In [32]:
from collections import Counter

result_li = Counter(li_list)

In [33]:
result_li

Counter({'三十': 19,
         '離宮': 3,
         '樓臺': 5,
         '與天': 9,
         '閣道': 3,
         '步行': 2,
         '美人': 28,
         '愁煙': 1,
         '恩疎': 4,
         '不及': 13,
         '桃李': 42,
         '傷春': 2,
         '淫樂': 3,
         '意何極': 2,
         '金輿': 3,
         '回中': 3,
         '萬乘': 16,
         '黃道': 2,
         '千旗': 2,
         '彩虹': 5,
         '前軍': 2,
         '細柳': 4,
         '後騎': 2,
         '甘泉': 5,
         '豈問': 2,
         '渭川': 4,
         '寧邀': 2,
         '野童': 2,
         '秋暮': 1,
         '瑤池': 7,
         '歸來': 26,
         '未窮': 5,
         '洗兵': 2,
         '條支': 2,
         '海上': 21,
         '放馬': 2,
         '天山': 8,
         '雪中': 4,
         '萬里': 60,
         '長征': 2,
         '三軍': 11,
         '衰老': 5,
         '秦家': 5,
         '築城': 2,
         '備胡處': 1,
         '漢家': 13,
         '還有': 4,
         '烽火': 10,
         '不息': 4,
         '征戰': 8,
         '無已': 2,
         '野戰': 2,
         '格鬬': 2,
         '敗馬': 2,
         '號鳴': 

In [34]:
result_li.most_common(10)

[('不可', 75),
 ('黃金', 68),
 ('春風', 65),
 ('萬里', 60),
 ('白日', 57),
 ('明月', 52),
 ('青天', 49),
 ('相思', 49),
 ('天地', 47),
 ('青雲', 44)]

In [35]:
def tang_author_topK(author,K):
    author_seg = tang_df[tang_df['author']==author]['dic_total'].tolist()
    author_list = reduce(operator.add, author_seg) 
    result_author = Counter(author_list)
    return  result_author.most_common(K)

In [36]:
tang_author_topK('李白',10)

[('不可', 75),
 ('黃金', 68),
 ('春風', 65),
 ('萬里', 60),
 ('白日', 57),
 ('明月', 52),
 ('青天', 49),
 ('相思', 49),
 ('天地', 47),
 ('青雲', 44)]

In [37]:
def song_author_topK(author,K):
    author_seg = song_df[song_df['author']==author]['dic_total'].tolist()
    author_list = reduce(operator.add, author_seg) 
    result_author = Counter(author_list)
    return  result_author.most_common(K)

In [38]:
song_author_topK('苏轼',10)

[('归去', 44),
 ('何处', 39),
 ('使君', 28),
 ('佳人', 26),
 ('人间', 23),
 ('明月', 22),
 ('江南', 20),
 ('尊前', 20),
 ('千里', 19),
 ('风流', 18)]

### 唐朝诗人关系网络创建，并存储在save文件夹中

In [39]:
import pickle
import sqlite3
from collections import Counter, defaultdict

In [40]:
def combine_list(list_df):
    s=''
    return s.join(list_df)

In [41]:
tang_df['paragraphs_t']=tang_df['paragraphs'].apply(lambda x:combine_list(x))

In [43]:
# TODO 补充著名诗人列表
# 這些詩人在CBDB的重名难以轻易排除，手動查找其在BIOG_MAIN表中的ID
# 注意CBDB使用的是繁體中文
manual_defuzzy_authors_id = {
    '李林甫': 32534, '王建': 92047,
    '李賀': 93012, '張繼': 93495,
    '張旭': 93409, '李紳': 92982}
# 手動刪除某些作者
mannual_deleted_authors = set(['無作', '清江'])
# 手動刪除作者的某些別稱，这些别称在唐诗中是常用字
mannual_deleted_alter_names = {'李林甫': set(['李十']),
    '李益': set(['李十']),
    '李世民': set(['李二']),
    '李嘉祐': set(['李二']),
    '馬湘': set(['自然']),
    '高駢': set(['千里']),
    '孟浩然': set(['浩然']),
    '李白': set(['太白']),
    '黃巢': set(['皇帝']),
    '眉娘': set(['逍遙'])}
# 補充CBDB中缺少的部分作者別稱
mannual_added_alter_names = {
    '李建': set(['李十一']),
    '劉禹錫': set(['劉二十八'])
    }

In [44]:
def read_poem(p_dataframe, p_type='tang'):
    if p_type=='tang':
        authors_set=set(p_dataframe['author'])
        poem_list= p_dataframe[['author','title','paragraphs_t']].values.tolist()
    if p_type=='song':
        authors_set=set(p_dataframe['author'])
        poem_list= p_dataframe[['author','rhythmic','paragraphs_t']].values.tolist()
    return authors_set,poem_list

In [58]:
def get_alter_names(qts_file, cbdb_file, save_dir,p_type='tang'):
    alter_names_file = os.path.join(save_dir, "alternames.pkl")

    if os.path.exists(alter_names_file):
        print("find dumped alternames file, loading directly.")
        with open(alter_names_file, 'rb') as f:
            qts_list, authors_filtered_by_CBDB, alter_names_dict = pickle.load(f)
    else:
        print("processing QuanTangShi...")
        # 读取全唐诗，并存储诗歌内容和作者
        authors_set, qts_list  = read_poem(qts_file,p_type)
        # 删除部分作者
        authors_set = authors_set - mannual_deleted_authors

        alter_names_dict, authors_filtered_by_CBDB = get_alter_names_from_CBDB(cbdb_file, authors_set,
                                                                               manual_defuzzy_authors_id,p_type)
        # 刪除不想要的別稱
        for k, v in mannual_deleted_alter_names.items():
            alter_names_dict[k] -= v
        # 補充CBDB中缺少的別稱
        for k, v in mannual_added_alter_names.items():
            alter_names_dict[k] |= v

        # 存储计算结果
        with open(alter_names_file, 'wb') as f:
            pickle.dump([qts_list, authors_filtered_by_CBDB, alter_names_dict], f)

    return qts_list, authors_filtered_by_CBDB, alter_names_dict

In [46]:
def get_refer_relations(qts_list, authors_filtered_by_CBDB, alter_names_dict, save_dir):
    reference_relations_file = os.path.join(save_dir, 'reference_relations.pkl')

    if os.path.exists(reference_relations_file):
        print("find dumped reference relations file, skip calculating.")
        return
    else:
        print("calculating reference relations...")
    reference_relations_counter = Counter()
    reference_relations_text = defaultdict(list)
    # 逐个作者搜寻
    for name in authors_filtered_by_CBDB:
        # 逐首诗搜寻
        for author, title, text in qts_list:
            # 如果不在CBDB过滤过的set中，直接跳过
            if author not in authors_filtered_by_CBDB:
                continue
            poem = title + ' ' + text
        
            # 查找本名，标题加正文中只要出现一次名字就可以
            if poem.find(name) != -1:
                reference_relations_counter[(author, name)] += 1
                reference_relations_text[(author, name)].append(title)
                continue
            # 查找别名
            alt_names = alter_names_dict[name]
            for alt_name in alt_names:
                if poem.find(alt_name) != -1:
                    reference_relations_counter[(author, name)] += 1
                    reference_relations_text[(author, name)].append(title)
                    break
    # 存储计算结果
    with open(reference_relations_file, 'wb') as f:
        pickle.dump([reference_relations_counter, reference_relations_text], f)

In [47]:
# 从CBDB中获取诗人们的别名
def get_alter_names_from_CBDB(db_file, authors_set, manual_defuzzy_authors_id,p_type='tang'):
    if p_type=='tang':
        tang_begin_year = 618  # 唐朝建立年份
        tang_end_year = 907  # 唐朝灭亡年份
    if p_type=='song':
        tang_begin_year = 960  # 唐朝建立年份
        tang_end_year = 1279  # 唐朝灭亡年份

    # 手动排查的诗人集合
    mannual_defuzzy_authors = set(manual_defuzzy_authors_id.keys())

    authors_not_in_CBDB = set()
    fuzzy_authors = set()
    fuzzy_authors_details = {}
    alter_names_dict = defaultdict(set)

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    for author in authors_set:
        # 如果在手动排查集合中，直接使用
        if author in mannual_defuzzy_authors:
            author_id = manual_defuzzy_authors_id[author]
        else: # 否则从CBDB中查询
          # import ipdb; ipdb.set_trace()
          # 某些诗人的名字在全唐诗中和CBDB中不一致，用模糊搜索更好
          # 比如"贯休"在CBDB中的名字为"释贯休"
            author_pattern = '%' + author
            cursor.execute('SELECT c_personid, c_birthyear, c_deathyear FROM BIOG_MAIN WHERE c_name_chn LIKE?',
                         (author_pattern,))
            person_info_list = cursor.fetchall()

      # 排除重名现象
      # 具体策略请参考我的微信公众号(mrqianjinsi)文章《计算机告诉你，唐朝诗人之间的关系到底是什么样的？》
        candidate_author_ids = []
        # import ipdb; ipdb.set_trace()
        for person_id, birth_year, death_year in person_info_list:
            if birth_year and death_year:  # 生卒年俱全
                if birth_year < tang_end_year and death_year > tang_begin_year:
                # 一旦找到一个生卒年俱全且和唐朝有交集的，就不看其他的了
                    candidate_author_ids = [person_id]
                    break
            elif birth_year or death_year:  # 只有生年或者卒年
                year = birth_year if birth_year else death_year
                if year > tang_begin_year and year < tang_end_year:
                    candidate_author_ids.append(person_id)

        # 候选名单为空或者多于一个人的候选名单都不要
        if not candidate_author_ids:
            authors_not_in_CBDB.add(author)
            # print('can\'t find valid items for %s' % author)
            continue
        elif len(candidate_author_ids) > 1:
            fuzzy_authors.add(author)
            fuzzy_authors_details[author] = candidate_author_ids
            # print('fuzzy authors: %s' % author)
            continue

        author_id = candidate_author_ids[0]

    # 根据author_id找出诗人别名
    cursor.execute('SELECT c_alt_name_chn FROM ALTNAME_DATA WHERE c_personid=?',
                   (author_id,))
    alt_name_list = cursor.fetchall()
    for alt_name in alt_name_list:
      # 不要只有一个字的别称
      if len(alt_name[0]) > 1:
        alter_names_dict[author].add(alt_name[0])

    conn.close()

  # 经过CBDB过滤过的诗人，接下来只分析这些人之间的关系
    authors_filtered_by_CBDB = authors_set - authors_not_in_CBDB - fuzzy_authors

    return alter_names_dict, authors_filtered_by_CBDB

In [48]:
qts_list, authors_filtered_by_CBDB, alter_names_dict = get_alter_names(tang_df, 'data/cbdb_sqlite.db', 'save')
get_refer_relations(qts_list, authors_filtered_by_CBDB, alter_names_dict, 'save')

find dumped alternames file, loading directly.
calculating reference relations...


### 读取网络进行可视化

In [55]:
import os
import math

# 如果需要，可以用opencc实现繁体和简体之间的转换
# 需要在电脑上安装opencc
# opencc = 'opencc -i echart_visualize/poets_network_early.html -o echart_visualize/poets_network_early_zhs.html -c zht2zhs.ini'


# 直接获取排名前visulize_range的引用关系
def get_concerned_relations_by_range(reference_relations_counter, visualize_range):
    # 获取引用关系
    relations = reference_relations_counter.most_common(visualize_range)

    max_refer_count = relations[0][1]
    min_refer_count = relations[-1][1]

    return relations, max_refer_count, min_refer_count

# 获取指定诗人群体之间的引用关系，适合画出某个群体内部的网络
def get_concerned_relations_by_authors(reference_relations_counter, authors):
    # 获取指定作者群体内部的引用关系
    relations = []
    max_refer_count = 0
    min_refer_count = 10000
    for (refered_by, refered), count in reference_relations_counter.items():
        # 不统计自引用的count
        if refered_by == refered:
            continue
        if refered_by in authors and refered in authors:
            if count > max_refer_count:
                max_refer_count = count
            if count < min_refer_count:
                min_refer_count = count

            relations.append(((refered_by, refered), count))

    return relations, max_refer_count, min_refer_count

# 有些时候如果画出所有关系会显得非常拥挤，用count_to_plot_threshold来控制最小显示出来的关系
# 只有引用数大于等于count_to_plot_threshold的关系才会显示出来
def generate_html_page(relations, max_refer_count, min_refer_count, saved_html_file, count_to_plot_threshold = 1):
    html_dir = os.path.dirname(saved_html_file)
    html_head_path = os.path.join(html_dir, 'html_head.txt')
    html_tail_path = os.path.join(html_dir, 'html_tail.txt')

    min_link_width = 0.5
    max_link_width = 3.0

    # 因为引用关系的强弱范围很大，对其开方降低变化范围，画图更直观
    max_refer_count = math.sqrt(max_refer_count)
    min_refer_count = math.sqrt(min_refer_count)
    width_slope = (max_link_width - min_link_width) / (max_refer_count - min_refer_count)
    # 格式化links数据
    links_text = 'links: [\n'
    links_item_format = """{source: '%s', target: '%s',
    lineStyle:{normal:{width: %f}}},
    """
    filtered_authors = set()
    for (refered_by, refered), count in relations:
        # 跳过自引用，不然有可能画出孤立节点
        if refered_by == refered:
            continue
        # 小于门限跳过
        if count < count_to_plot_threshold:
            continue

        filtered_authors.add(refered_by)
        filtered_authors.add(refered)
        count = math.sqrt(count)
        line_width = min_link_width + width_slope * (count - min_refer_count)
        links_text += links_item_format % (refered_by, refered, line_width)

    links_text += '],\n'

    # 格式化node数据
    data_text = 'data:[\n'
    data_item_format = "{name: '%s'},\n"
    for author in filtered_authors:
        data_text += data_item_format % author

    data_text += '],\n'

    # 读取html的head和tail部分
    with open(html_head_path, 'r', encoding = 'utf-8') as f:
        head_text = f.read()

    with open(html_tail_path, 'r', encoding = 'utf-8') as f:
        tail_text = f.read()

    # 合并存储为html
    with open(saved_html_file, 'w', encoding = 'utf-8') as f:
        f.write(head_text + data_text + links_text + tail_text)

In [56]:
with open('save/reference_relations.pkl', 'rb') as f:
        
        reference_relations_counter, reference_relations_text = pickle.load(f)
    
# 生成全唐排名前100的关系图
relations, max_refer_count, min_refer_count = get_concerned_relations_by_range(reference_relations_counter, 100)
saved_html = os.path.join('html', 'full_tang_poets_net.html')
generate_html_page(relations, max_refer_count, min_refer_count, saved_html)

# 生成初唐、盛唐、中唐、晚唐四个时期的诗人关系图
#                      诗人名字文件              社交关系图网页              引用数门限
files_name_array = [('early_tang_poets.txt', 'early_tang_poets_net.html',  1),
                  ('high_tang_poets.txt',  'high_tang_poets_net.html',   2),
                  ('middle_tang_poets.txt','middle_tang_poets_net.html', 2),
                  ('late_tang_poets.txt',  'late_tang_poets_net.html',   1)]

for authors_file_name, html_file_name, threshold in files_name_array:
    authors_file_path = os.path.join('data', authors_file_name)
    with open(authors_file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    authors = set(text.split())

    relations, max_refer_count, min_refer_count = get_concerned_relations_by_authors(reference_relations_counter, authors)

    saved_html = os.path.join("html", html_file_name)
    generate_html_page(relations, max_refer_count, min_refer_count, saved_html, threshold)




### 宋朝诗人网络创建

In [61]:
qts_list, authors_filtered_by_CBDB, alter_names_dict = get_alter_names(song_df, 'data/cbdb_sqlite.db', 'song_save','song')
get_refer_relations(qts_list, authors_filtered_by_CBDB, alter_names_dict, 'song_save')

processing QuanTangShi...
calculating reference relations...


In [65]:
with open('song_save/reference_relations.pkl', 'rb') as f:
        
        reference_relations_counter, reference_relations_text = pickle.load(f)
    
# 生成全宋排名前100的关系图
relations, max_refer_count, min_refer_count = get_concerned_relations_by_range(reference_relations_counter, 100)
saved_html = os.path.join('html', 'full_song_poets_net.html')
generate_html_page(relations, max_refer_count, min_refer_count, saved_html)